Reaktoro is a unified framework for modeling chemically reactive systems.

Copyright © 2014-2021 Allan Leal

This library is free software; you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public
License as published by the Free Software Foundation; either
version 2.1 of the License, or (at your option) any later version.

This library is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public License
along with this library. If not, see <http://www.gnu.org/licenses/>.

-----------------------------------------------------------------------------
👏 Acknowledgements 👏
-----------------------------------------------------------------------------
This example was originally authored by:
  • Allan Leal (16 July 2021)

and since revised by:
  • Allan Leal (22 July 2021)
-----------------------------------------------------------------------------

In [ ]:
from reaktoro import *

In [ ]:
db = SupcrtDatabase("supcrtbl")

In [ ]:
solution = AqueousPhase(speciate("Na Cl C Ca Mg Si"), exclude("organic"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

In [ ]:
gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelPengRobinson())

In [ ]:
minerals = MineralPhases("Halite Calcite Magnesite Dolomite Quartz")

In [ ]:
system = ChemicalSystem(db, solution, gases, minerals)

In [ ]:
statex = ChemicalState(system)
statex.temperature(60.0, "celsius")
statex.pressure(100.0, "bar")
statex.set("H2O(aq)"  , 1.00, "kg")
statex.set("Halite"   , 1.00, "mol")
statex.set("Calcite"  , 1.00, "mol")
statex.set("Magnesite", 1.00, "mol")
statex.set("Quartz"   , 1.00, "mol")

In [ ]:
equilibrate(statex)

In [ ]:
propsx = ChemicalProps(statex)
Vx = propsx.volume()
Ux = propsx.internalEnergy()

In [ ]:
statex.output("state-expected.txt")
propsx.output("props-expected.txt")

In [ ]:
specs = EquilibriumSpecs(system)

In [ ]:
idxV = specs.addInput("V")
idxU = specs.addInput("U")

In [ ]:
volumeConstraint = ConstraintEquation()
volumeConstraint.id = "VolumeConstraint"
volumeConstraint.fn = lambda props, w: props.volume() - w[idxV]

In [ ]:
internalEnergyConstraint = ConstraintEquation()
internalEnergyConstraint.id = "InternalEnergyConstraint"
internalEnergyConstraint.fn = lambda props, w: props.internalEnergy() - w[idxU]

In [ ]:
specs.addConstraint(volumeConstraint)
specs.addConstraint(internalEnergyConstraint)

In [ ]:
conditions = EquilibriumConditions(specs)
conditions.set("V", Vx)
conditions.set("U", Ux)
conditions.setLowerBoundPressure(1.0, "bar")

In [ ]:
state = ChemicalState(system)
state.temperature(25.0, "celsius")
state.pressure(1.0, "bar")
state.set("H2O(aq)"  , 1.00, "kg")
state.set("Halite"   , 1.00, "mol")
state.set("Calcite"  , 1.00, "mol")
state.set("Magnesite", 1.00, "mol")
state.set("Quartz"   , 1.00, "mol")

In [ ]:
solver = EquilibriumSolver(specs)

In [ ]:
solver.solve(state, conditions)

In [ ]:
props = ChemicalProps(state)

In [ ]:
state.output("state.txt")
props.output("props.txt")

Check if props.txt and props-expected.txt are numerically equivalent.